# 멋진 인공지능 작사가 만들기

작곡언어는 (아쉽게도)영어입니다. 다음과 같은 순서로 만들어 볼 수 있습니다.😊

> **Step 1. 데이터 다운로드  
Step 2. 데이터 읽어오기  
Step 3. 데이터 정제  
Step 4. 평가 데이터셋 분리  
Step 5. 인공지능 만들기**

그럼 시작해볼까요?

### Step 1. 데이터 다운로드

> 먼저 아래 링크에서 Song Lyrics 데이터를 다운로드해 주세요!<br>저장된 파일을 압축 해제한 후, 모든 txt 파일을 lyrics 폴더를 만들어 그 속에 저장해주세요!<br>아니면 그 아래의 명령어를 실행하셔도 됩니다.

- https://www.kaggle.com/paultimothymooney/poetry/data

1. wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
2. unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics  <span style="color:green">#lyrics 폴더에 압축풀기</span>

### Step 2. 데이터 읽어오기

glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이해요.  
glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하도록 할게요!

In [1]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import glob                # 파일들의 리스트를 뽑을 때 사용
import os                  # 운영 체제의 환경 변수 값을 읽어올 수 있음!

# 파일을 읽기모드로 열어 봅니다.
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'  # * : all
txt_list = glob.glob(txt_file_path)

raw_corpus = []  # 여러개의 txt 파일을 모두 읽어서 raw_corpus에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['', '', "Jesus died for somebody's sins but not mine"]


### Step 3. 데이터 정제

##### 이번 스텝에서 데이터셋을 생성하기 위해 거칠 과정입니다. 

>- 정규표현식을 이용한 corpus 생성
>- tf.keras.preprocessing.text.Tokenizer를 이용해 corpus를 텐서로 변환
>- tf.data.Dataset.from_tensor_slices()를 이용해 corpus 텐서를 tf.data.Dataset객체로 변환

dataset을 얻으면 데이터 다듬기 과정은 끝납니다.<br>
tf.data.Dataset에서 제공하는 shuffle(), batch() 등 다양한 데이터셋 관련 기능을 손쉽게 이용할 수 있게 되었군요.

이 모든 일련의 과정을 텐서플로우에서의 **데이터 전처리**라 칭합니다!

**문장 생성에 적합한 모양새로 데이터를 정제하세요!**

preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래가사 작사하기에 어울리지 않을수도 있겠죠.
그래서 이번에는 문장을 **토큰화** 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외하겠습니다.

가장 심플한 방법은 띄어쓰기를 기준으로 나누는 방법이지만 이 방법에는 몇가지 문제가 있습니다.

##### 몇 가지 문제 케이스
1. Hi, my name is John. *("Hi," "my", …, "john." 으로 분리됨) -문장부호
2. First, open the first chapter. *(First와 first를 다른 단어로 인식) -대소문자
3. He is a ten-year-old boy. *(ten-year-old를 한 단어로 인식) -특수문자

"1." 을 막기 위해 문장 부호 양쪽에 공백을 추가 할 거고요,<br>
"2." 를 막기 위해 모든 문자들을 소문자로 변환할 겁니다.<br>
"3." 을 막기 위해 특수문자들은 모두 제거하도록 하죠!

이런 전처리를 위해 정규표현식(Regex)을 이용한 필터링이 유용하게 사용됩니다.

In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


짜잔, 지저분한 문장을 넣어도 예쁘게 변환해주는 정제 함수가 완성되었습니다!<br>보너스로 start, end 도 추가했습니다.  

자연어처리 분야에서 모델의 입력이 되는 문장을 소스 문장(Source Sentence), 정답 역할을 하게 될 모델의 출력 문장을 타겟 문장(Target Sentence)라고 관례적으로 부릅니다. 각각 X_train, y_train 에 해당한다고 할 수 있겠죠?

그렇다면 우리는 위에서 만든 정제 함수를 통해 만든 데이터셋에서 토큰화를 진행한 후 끝 단어 <end>를 없애면 소스 문장, 첫 단어 <start>를 없애면 타겟 문장이 되겠죠? 이 정제 함수를 활용해서 아래와 같이 정제 데이터를 구축합니다!

In [3]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0 or len(sentence) > 15: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

print(sentence)

or how things used to be And I had to tell Willy every look stand out 


이제 데이터는 완벽하게 준비가 된 것 같네요!<br>

텐서플로우는 자연어 처리를 위한 여러 가지 모듈을 제공하는데, 우리도 그 모듈을 십분 활용할 겁니다!<br>
아래에서 활용하게 될 **tf.keras.preprocessing.text.Tokenizer 패키지**는 정제된 데이터를 토큰화하고, 단어 사전(vocabulary 또는 dictionary라고 칭함)을 만들어주며, 데이터를 숫자로 변환까지 한 방에 해줍니다.<br>
이 과정을 벡터화(vectorize) 라 하며, 숫자로 변환된 데이터를 텐서(tensor) 라고 칭합니다.<br>
우리가 사용하는 텐서플로우로 만든 모델의 입출력 데이터는 실제로는 모두 이런 텐서로 변환되어 처리되는 것입니다.

In [4]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   15 1908 ...    0    0    0]
 [   2   70   56 ...    0    0    0]
 [   2   70   56 ...    0    0    0]
 ...
 [   2 1001 3416 ...    0    0    0]
 [   2  138  138 ...    0    0    0]
 [   2  138  138 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7ff0c2883b90>


텐서 데이터는 모두 정수로 이루어져 있습니다.<br>
이 숫자는 다름 아니라, tokenizer에 구축된 단어 사전의 인덱스입니다.<br>
단어 사전이 어떻게 구축되었는지 아래와 같이 확인해 봅시다.

In [5]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : .
7 : you
8 : oh
9 : it
10 : me


어떻습니까? 2번 인덱스가 바로 **start**였습니다.<br>
왜 모든 행이 2로 시작하는지 이해할 수 있겠습니다.

이제 생성된 텐서를 소스와 타겟으로 분리하여 모델이 학습할 수 있게 하겠습니다.<br>
이 과정도텐서플로우가 제공하는 모듈을 사용할 것이니, 어떻게 사용하는지만 눈여겨 봐둡시다.

텐서 출력부에서 행 뒤쪽에 0이 많이 나온 부분은 정해진 입력 시퀀스 길이보다 문장이 짧을 경우 0으로 패딩(padding)을 채워넣은 것입니다.<br>
사전에는 없지만 0은 바로 패딩 문자 **pad**가 될 것입니다.

In [6]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2   15 1908   15  522    3    0    0    0    0    0    0]
[  15 1908   15  522    3    0    0    0    0    0    0    0]


마지막으로 데이터셋 객체를 생성할 것입니다.<br>
텐서플로우를 활용할 경우 텐서로 생성된 데이터를 이용해 tf.data.Dataset객체를 생성하는 방법을 흔히 사용합니다.<br>
tf.data.Dataset객체는 텐서플로우에서 사용할 경우 데이터 입력 파이프라인을 통한 속도 개선 및 각종 편의기능을 제공하므로 꼭 사용법을 알아 두는것을 권장합니다.<br>
이미 위에서 데이터셋을 텐서 형태로 생성해 두었으므로, tf.data.Dataset.from_tensor_slices() 메소드를 이용해 tf.data.Dataset객체를 생성할 것입니다.

In [7]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 12), (256, 12)), types: (tf.int32, tf.int32)>

### Step 4. 평가 데이터셋 분리

**훈련 데이터와 평가 데이터를 분리하세요!**

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다.  
단어장의 크기는 12,000 이상으로 설정하세요!  

총 데이터의 **20%** 를 평가 데이터셋으로 사용하겠습니다.

In [8]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (11296, 12)
Target Train: (11296, 12)


만약 학습데이터 갯수가 124960보다 크다면 위 Step 3.의 데이터 정제 과정을 다시한번 검토해 보세요!

### Step 5. 인공지능 만들기

모델의 **Embedding Size**와 **Hidden Size**를 조절하며 **10 Epoch** 안에 **val_loss 값을 2.2 수준**으로 줄일 수 있는 모델을 설계하세요!  
(Loss는 아래 제시된 Loss 함수를 그대로 사용!)

In [9]:
# Loss 함수

# loss = tf.keras.losses.SparseCategoricalCrossentropy(
#     from_logits=True, reduction='none')

"""""", '''''' 로 줄 주석처리 하려니 안되네요..  
주석을 하고 싶은 부분들을 드래그 해서 영역을 선택한 후 Ctrl + / 를 누르면 된다길래 해보니 #으로 줄마다 주석처리 됩니다.🤔🤔🤔
이 부분은 추후 다시 알아보기로.....🧐

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 12, 7001), dtype=float32, numpy=
array([[[-1.5630612e-04, -2.1479298e-04,  4.7894078e-04, ...,
         -5.5313401e-04, -5.6747791e-05,  1.4602030e-05],
        [-2.0139404e-04, -6.6264608e-04,  1.8932828e-04, ...,
         -1.1565241e-03,  1.5133073e-05, -1.6830349e-04],
        [-2.5849909e-04, -7.7325472e-04,  1.2038055e-05, ...,
         -1.0125804e-03,  1.8405824e-04, -5.3904357e-04],
        ...,
        [ 3.5242317e-03,  1.7246478e-03,  8.5587282e-04, ...,
         -1.9012154e-03,  2.7056127e-05,  9.5006684e-04],
        [ 4.1646911e-03,  2.1634460e-03,  1.1990423e-03, ...,
         -2.0568324e-03,  5.3903928e-05,  1.1703369e-03],
        [ 4.7409656e-03,  2.5637052e-03,  1.5439735e-03, ...,
         -2.1806376e-03,  5.6141813e-05,  1.3372115e-03]],

       [[-1.5630612e-04, -2.1479298e-04,  4.7894078e-04, ...,
         -5.5313401e-04, -5.6747791e-05,  1.4602030e-05],
        [-5.1237922e-04, -3.9897300e-04,  6.8201660e-04, ...,
         -7.6508749e-04, -

ResourceExhaustedError: OOM when allocating tensor with shape[88576,7001] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:MatMul] 에러가 발목을 잡는당.....ㅠㅠㅠㅠ

model의 input shape가 결정되면서 model.build()가 자동으로 호출됩니다.

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


이제 모델이 학습할 준비가 완료되었습니다.<br>
아래 코드를 실행해 모델을 학습시켜보세요!<br>
학습엔 10분 정도 소요됩니다(GPU 환경 기준)<br>
혹시라도 학습에 지나치게 많은 시간이 소요된다면 tf.test.is_gpu_available() 소스를 실행해 텐서플로우가 GPU를 잘 사용하고 있는지 확인하시길 바랍니다!

In [19]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
55/55 [==============================] - 5s 90ms/step - loss: 0.8971
Epoch 2/10
55/55 [==============================] - 5s 90ms/step - loss: 0.8626
Epoch 3/10
55/55 [==============================] - 5s 86ms/step - loss: 0.8490
Epoch 4/10
55/55 [==============================] - 5s 89ms/step - loss: 0.8393
Epoch 5/10
55/55 [==============================] - 5s 86ms/step - loss: 0.8288
Epoch 6/10
55/55 [==============================] - 5s 85ms/step - loss: 0.8208
Epoch 7/10
55/55 [==============================] - 5s 84ms/step - loss: 0.8130
Epoch 8/10
55/55 [==============================] - 5s 89ms/step - loss: 0.8061
Epoch 9/10
55/55 [==============================] - 5s 88ms/step - loss: 0.7990
Epoch 10/10
55/55 [==============================] - 5s 88ms/step - loss: 0.7933


In [14]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]  
 
        test_tensor = tf.concat([test_tensor, 
                                tf.expand_dims(predict_word, axis=0)], axis=-1)
        
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 문장 반환

In [15]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

모델이 생성한 가사 한 줄을 확인해보겠습니다.

In [18]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=15)

'<start> i love you <end> '

라 그럴듯하죠? ㅎㅎㅎ

## 루브릭

1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?

텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가? 넵!

2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?

특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가? 넵!

3. 텍스트 생성모델이 안정적으로 학습되었는가?

텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?

## 회고

빡센 저번주의 노드보다 훨씬 간결해서 이게 맞나...? 했는데 조원들과 얘기해보니 저번주 노드과제들이 난도가 높았다는 걸 새삼 느꼈다.  
아무래도 🇰🇷한국어보다 공개된 🇺🇸영어데이터가 많아 아쉽긴하지만 이번주 노드는 좀 더 재밌게 할 수 있었다.  
다만 영어공부도 다시 해야...할 필요성을 계속 느끼고 있어서 계획을 세워봐야겠다.